In [1]:
import functools
import logging
import os
import subprocess
import warnings

import hydra
import hydra.core.hydra_config
import omegaconf

import pathlib
import typing


#Register additional resolver for log path
omegaconf.OmegaConf.register_new_resolver("list_to_string", lambda o: functools.reduce(lambda acc, x: acc+", "+x.replace("\"","").replace("/"," "), o, "")[2:])
omegaconf.OmegaConf.register_new_resolver("eval", lambda c: eval(c))

import pytorch_lightning
import pytorch_lightning.accelerators
import pytorch_lightning.callbacks
import pytorch_lightning.loggers
import pytorch_lightning.utilities

import torch
import torch.utils.tensorboard
import torch.version

from mnist_datamodule import MNISTDataModule
from ellipses_datamodule import EllipsesDataModule
from learned_filter_model import LearnedFilterModel
from analytic_filter_model import AnalyticFilterModel
from learned_svd_model import LearnedSVDModel


#Custom version of pytorch lightnings TensorBoardLogger, to allow manipulation of internal logging settings
class CustomTensorBoardLogger(pytorch_lightning.loggers.TensorBoardLogger):
    #Disables logging of epoch
    @pytorch_lightning.utilities.rank_zero_only
    def log_metrics(self, metrics: dict[str, typing.Union[torch.Tensor, float]], step: int) -> None:
        metrics.pop("epoch", None)
        return super().log_metrics(metrics, step)
    
    #Disables creation of hparams.yaml
    @pytorch_lightning.utilities.rank_zero_only
    def save(self) -> None:
        dir_path = self.log_dir
        if not os.path.isdir(dir_path):
            dir_path = self.save_dir



In [2]:
@hydra.main(config_path="configs", config_name="default", version_base=None)
def main(config: omegaconf.DictConfig) -> None:
    #Setup logging
    logger = logging.getLogger(__name__)
    logging.captureWarnings(True)
    logging.getLogger("pytorch_lightning").handlers.append(logger.root.handlers[1]) #Route pytorch lightning logging to hydra logger
    for old_log in os.listdir(hydra.core.hydra_config.HydraConfig.get().runtime.output_dir):
        if old_log.startswith("events.out.tfevents"):
            os.remove(os.path.join(hydra.core.hydra_config.HydraConfig.get().runtime.output_dir, old_log))

    #Set num_workers to available CPU count
    if config.num_workers == -1:
        config.num_workers = os.cpu_count()
    
    #Initialize determinism
    if config.deterministic:
        pytorch_lightning.seed_everything(config.seed, workers=True)

    #Create model and load data
    if config.model.name == "analytic":
        modelClass = AnalyticFilterModel
    elif config.model.name == "learned":
        modelClass = LearnedFilterModel
    elif config.model.name == "svd":
        modelClass = LearnedSVDModel
    else:
        raise NotImplementedError()
    if config.checkpoint != None:
        model = modelClass.load_from_checkpoint(os.path.abspath(os.path.join("../../" if hydra.core.hydra_config.HydraConfig.get().mode == hydra.types.RunMode.MULTIRUN else "../", config.checkpoint)), config=config)
    else:
        model = modelClass(config)
    if config.dataset.name == "MNIST":
        datamodule = MNISTDataModule(config)
    elif config.dataset.name == "ellipses":
        datamodule = EllipsesDataModule(config)
    else:
        raise NotImplementedError()
    
    
    datamodule = EllipsesDataModule(config)
    return datamodule

In [5]:
datamodule = main()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: argument --shell-completion/-sc: ignored explicit argument '9002'


AssertionError: 

In [31]:
@hydra.main(config_path="configs", config_name="default", version_base=None)
def main(config: omegaconf.DictConfig) -> None:
    #Setup logging
    logger = logging.getLogger(__name__)
    logging.captureWarnings(True)
    logging.getLogger("pytorch_lightning").handlers.append(logger.root.handlers[1]) #Route pytorch lightning logging to hydra logger
    for old_log in os.listdir(hydra.core.hydra_config.HydraConfig.get().runtime.output_dir):
        if old_log.startswith("events.out.tfevents"):
            os.remove(os.path.join(hydra.core.hydra_config.HydraConfig.get().runtime.output_dir, old_log))

    #Set num_workers to available CPU count
    if config['num_workers'] == -1:
        config['num_workers'] = os.cpu_count()
    
    #Initialize determinism
    if config['deterministic']:
        pytorch_lightning.seed_everything(config.seed, workers=True)

    #Create model and load data
    if config['model']['name'] == "analytic":
        modelClass = AnalyticFilterModel
    elif config['model']['name'] == "learned":
        modelClass = LearnedFilterModel
    elif config['model']['name'] == "svd":
        modelClass = LearnedSVDModel
    else:
        raise NotImplementedError()
    if config['checkpoint'] != None:
        model = modelClass.load_from_checkpoint(os.path.abspath(os.path.join("../../" if hydra.core.hydra_config.HydraConfig.get().mode == hydra.types.RunMode.MULTIRUN else "../", config.checkpoint)), config=config)
    else:
        model = modelClass(config)
    if config['dataset']['name'] == "MNIST":
        datamodule = MNISTDataModule(config)
    elif config['dataset']['name'] == "ellipses":
        datamodule = EllipsesDataModule(config)
    else:
        raise NotImplementedError()
    
    
    # datamodule = EllipsesDataModule(config)
    return datamodule

In [32]:
datamodule = main(config)

IndexError: list index out of range

In [3]:
config = {
    'dataset': {
        'name': 'ellipses'
    },
    'num_workers': 1,
    'out_dir': '/LearnedRadonFilters',
    'deterministic': True,
    'seed': 1234567,
    'device': "cuda",
    'num_workers': 1,
    'defaults':{
        'model': 'analytic',
        'dataset': 'ellipses'
    }
}

In [23]:
config['dataset']['name']

'ellipses'

In [24]:
config['something new'] = 2

In [10]:
config

{'dataset': {'name': 'ellipses'},
 'num_workers': 1,
 'out_dir': '/LearnedRadonFilters',
 'deterministic': True,
 'seed': 1234567,
 'device': 'cuda',
 'defaults': {'model': 'analytic', 'dataset': 'ellipses'}}

In [11]:
config = None

In [2]:
import pickle

In [13]:
# Saving the objects:
print("Salvo le variabili")
with open('test_datamodule.pkl', 'wb') as f:
    pickle.dump(config, f)


Salvo le variabili


In [26]:
try:
    del config
except:
    print("config does not exist")

config does not exist


In [3]:
try:
    del config
except:
    print("config does not exist")

with open('./LearnedRadonFilters/_debug/test_datamodule.pkl', 'rb') as f:
    config = pickle.load(f)

config does not exist


In [5]:
config

In [4]:
import os

this_dir = os.getcwd()
print(this_dir)

c:\Users\danil\Desktop\oct2022\test


In [7]:
print(config)

In [20]:
print(type(config))

<class 'NoneType'>


In [1]:
#pyright: reportPrivateImportUsage=false, reportGeneralTypeIssues=false
import functools
import logging
import os
import subprocess
import warnings

import hydra
import hydra.core.hydra_config
import omegaconf

import pathlib
import typing


#Register additional resolver for log path
omegaconf.OmegaConf.register_new_resolver("list_to_string", lambda o: functools.reduce(lambda acc, x: acc+", "+x.replace("\"","").replace("/"," "), o, "")[2:])
omegaconf.OmegaConf.register_new_resolver("eval", lambda c: eval(c))

import pytorch_lightning
import pytorch_lightning.accelerators
import pytorch_lightning.callbacks
import pytorch_lightning.loggers
import pytorch_lightning.utilities

import torch
import torch.utils.tensorboard
import torch.version

from mnist_datamodule import MNISTDataModule
from ellipses_datamodule import EllipsesDataModule
from learned_filter_model import LearnedFilterModel
from analytic_filter_model import AnalyticFilterModel
from learned_svd_model import LearnedSVDModel

import pickle


#Custom version of pytorch lightnings TensorBoardLogger, to allow manipulation of internal logging settings
class CustomTensorBoardLogger(pytorch_lightning.loggers.TensorBoardLogger):
    #Disables logging of epoch
    @pytorch_lightning.utilities.rank_zero_only
    def log_metrics(self, metrics: dict[str, typing.Union[torch.Tensor, float]], step: int) -> None:
        metrics.pop("epoch", None)
        return super().log_metrics(metrics, step)
    
    #Disables creation of hparams.yaml
    @pytorch_lightning.utilities.rank_zero_only
    def save(self) -> None:
        dir_path = self.log_dir
        if not os.path.isdir(dir_path):
            dir_path = self.save_dir


with open('./LearnedRadonFilters/_debug/test_datamodule_1.pkl', 'rb') as f:
    datamodule = pickle.load(f)
with open('./LearnedRadonFilters/_debug/test_config_1.pkl', 'rb') as f:
    config = pickle.load(f)




In [141]:
import dill as pickle

In [5]:
with open('./LearnedRadonFilters/_debug/test_datamodule.pkl', 'rb') as f:
    datamodule = pickle.load(f)
with open('./LearnedRadonFilters/_debug/test_config.pkl', 'rb') as f:
    config = pickle.load(f)

In [2]:
#Initialize determinism
if config.deterministic:
    pytorch_lightning.seed_everything(config.seed, workers=True)

#Create model and load data
if config.model.name == "analytic":
    modelClass = AnalyticFilterModel
elif config.model.name == "learned":
    modelClass = LearnedFilterModel
elif config.model.name == "svd":
    modelClass = LearnedSVDModel
else:
    raise NotImplementedError()
if config.checkpoint != None:
    model = modelClass.load_from_checkpoint(os.path.abspath(os.path.join("../../" if hydra.core.hydra_config.HydraConfig.get().mode == hydra.types.RunMode.MULTIRUN else "../", config.checkpoint)), config=config)
else:
    model = modelClass(config)
if config.dataset.name == "MNIST":
    datamodule = MNISTDataModule(config)
elif config.dataset.name == "ellipses":
    datamodule = EllipsesDataModule(config)
else:
    raise NotImplementedError()

Global seed set to 1234567


In [13]:
datamodule

In [4]:
len(datamodule.train_dataloader())

for X in iter(datamodule.train_dataloader()):
    break

In [3]:
import matplotlib.pyplot as plt

In [7]:
print(X[0].shape)

for cont in range(len(X[0])):
    plt.imshow((X[0][cont]).view(256,256))
    plt.savefig(f"mygraph__1_{cont:02d}.png")
    if cont>10:
        break

torch.Size([32, 1, 256, 256])


In [4]:
from math import atan, cos, sin, tan
import typing

import omegaconf
import pytorch_lightning as pl
import torch
import torch.utils.data
import torchvision.transforms
import torchvision.transforms.functional
import matplotlib
matplotlib.use("agg")
import matplotlib.patches
import matplotlib.pyplot as plt
import numpy as np


class _EllipsesDataset(torch.utils.data.Dataset):
    def __init__(self, img_count: int, img_size: int, ellipses_count: int, ellipses_size: float, ellipses_size_min: float=1, transform: typing.Union[typing.Callable[[torch.Tensor],torch.Tensor], None]=None, number_ellipses_poisson=False):
        self.img_count = img_count
        self.img_size = img_size
        if number_ellipses_poisson == True:
            self.ellipses_count = torch.poisson(torch.full((img_count,), ellipses_count).to(torch.float32)).to(torch.int32)
        else:
            self.ellipses_count = (torch.full((img_count,), ellipses_count).to(torch.float32)).to(torch.int32)
        real_ellipses_count = self.ellipses_count.sum()
        self.ellipse_width_aa = torch.rand((real_ellipses_count,))*max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        self.ellipse_height_aa = torch.rand((real_ellipses_count,))*max(0.0, ellipses_size-ellipses_size_min)+ellipses_size_min
        self.ellipse_x_raw = torch.rand((real_ellipses_count,))
        self.ellipse_y_raw = torch.rand((real_ellipses_count,))
        self.ellipse_angle = torch.rand((real_ellipses_count,))*360.0
        self.ellipse_alpha = torch.rand((real_ellipses_count,))*0.9+0.1

        #self.ellipses_count = torch.ones((img_count,), dtype=torch.int32)
        #self.ellipse_width_aa = torch.full((img_count,), 1.0)#ellipses_size)
        #self.ellipse_height_aa = torch.full((img_count,), 1.0)#ellipses_size)
        #self.ellipse_x_raw = torch.full((img_count,), 0.5)
        #self.ellipse_y_raw = torch.full((img_count,), 0.5)
        #self.ellipse_angle = torch.zeros((img_count,))
        #self.ellipse_alpha = torch.ones((img_count,))

        self.transform = transform

        self.channel_selection_mode = "alpha"
        if self[0][0].sum() == self.img_size*self.img_size:
            self.channel_selection_mode = "noalpha"

    def __len__(self) -> int:
        return self.img_count

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, int]:
        fig = plt.figure(figsize=(self.img_size,self.img_size), dpi=1)
        ax = fig.add_axes([0.0,0.0,1.0,1.0])
        ellipse_func = lambda w, h, a, t: (w/2.0*cos(t)*cos(a)-h/2.0*sin(t)*sin(a), w/2.0*cos(t)*sin(a)+h/2.0*sin(t)*cos(a))
        prev_ellipses_count = self.ellipses_count[:idx].sum()
        for i in range(self.ellipses_count[idx]):
            e_idx = prev_ellipses_count+i
            args = (self.ellipse_width_aa[e_idx].item(), self.ellipse_height_aa[e_idx].item(), self.ellipse_angle[e_idx].item()/180.0*torch.pi)
            if self.ellipse_angle[idx] == 0.0:
                ellipse_width = self.ellipse_width_aa[e_idx]
                ellipse_height = self.ellipse_height_aa[e_idx]
            else:
                t = atan(-self.ellipse_height_aa[e_idx].item()*tan(self.ellipse_angle[e_idx].item()/180.0*torch.pi)/self.ellipse_width_aa[e_idx].item())
                ellipse_width = max(ellipse_func(*args, t)[0], ellipse_func(*args, t+torch.pi)[0])-min(ellipse_func(*args, t)[0], ellipse_func(*args, t+torch.pi)[0])
                t = atan(self.ellipse_height_aa[e_idx].item()/(tan(self.ellipse_angle[e_idx].item()/180.0*torch.pi)*self.ellipse_width_aa[e_idx].item()))
                ellipse_height = max(ellipse_func(*args, t)[1], ellipse_func(*args, t+torch.pi)[1])-min(ellipse_func(*args, t)[1], ellipse_func(*args, t+torch.pi)[1])
            ellipse_x = ellipse_width/2.0+self.ellipse_x_raw[e_idx].item()*(self.img_size-ellipse_width)
            ellipse_y = ellipse_height/2.0+self.ellipse_y_raw[e_idx].item()*(self.img_size-ellipse_height)
            ellipse = matplotlib.patches.Ellipse(xy=[ellipse_x, ellipse_y], width=self.ellipse_width_aa[e_idx].item(), height=self.ellipse_height_aa[e_idx].item(), angle=self.ellipse_angle[e_idx].item())
            ax.add_artist(ellipse)
            ellipse.set_clip_box(ax.bbox)
            ellipse.set_alpha(self.ellipse_alpha[e_idx].item())
            ellipse.set_facecolor("black")
            ellipse.set_edgecolor(None)
            ellipse.set_antialiased(False)
        ax.axis("off")
        ax.set_xlim(0.0, self.img_size)
        ax.set_ylim(0.0, self.img_size)
        fig.add_axes(ax)
        fig.canvas.draw()
        img = torch.from_numpy(np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8).copy())
        if self.channel_selection_mode == "noalpha":
            img = 1.0-torch.swapaxes(img.reshape(self.img_size,self.img_size,4), 0, 2).to(torch.float32)[3:4]/255.0
        elif self.channel_selection_mode == "alpha":
            img = torch.swapaxes(img.reshape(self.img_size,self.img_size,4), 0, 2).to(torch.float32)[0:1]/255.0
        else:
            raise NotImplementedError()
        plt.close()
        if self.transform != None:
            img = self.transform(img)
        return img, 0



In [9]:
config.dataset.ellipse_count = 10

In [27]:
config.dataset.ellipse_count

1

In [15]:
my_img_count = 1#640*32
my_image_size = 256#256
my_ellipses_count = 10

In [21]:
print(config.training_batch_count)
print(config.training_batch_size)
print(config.dataset.img_size)

1
1
256


In [5]:
config

{'model': {'name': 'analytic'}, 'dataset': {'name': 'ellipses', 'img_size': 256, 'ellipse_count': 10, 'ellipse_size': 50, 'ellipse_size_min': 10, 'blurred': False}, 'out_dir': './LearnedRadonFilters', 'deterministic': True, 'seed': 1234567, 'device': 'cuda', 'num_workers': 1, 'checkpoint': None, 'validation_split_percent': 20, 'training_batch_size': 32, 'shuffle_training_data': True, 'drop_last_training_batch': False, 'validation_batch_size': '${training_batch_size}', 'shuffle_validation_data': False, 'drop_last_validation_batch': False, 'test_batch_size': '${training_batch_size}', 'shuffle_test_data': False, 'drop_last_test_batch': True, 'sino_angles': None, 'sino_positions': None, 'noise_level': 1.0, 'optimizer_lr': 0.001, 'epochs': 3, 'training_batch_count': -1, 'validation_batch_count': -1, 'test_batch_count': -1}

In [8]:
print(config.training_batch_count     )
print(config.training_batch_size      )
print(config.dataset.img_size         )
print(config.dataset.ellipse_size     )
print(config.dataset.ellipse_size_min )
print(config.dataset.ellipse_count    )

-1
32
256
50
10
10


In [164]:
config.training_batch_count     = -1 #2    #-1
config.training_batch_size      = 1 # 32 #1    #32
config.dataset.img_size         = 64 #256 #256
config.dataset.ellipse_size     = 20 #5 #50
config.dataset.ellipse_size_min = 20 #5 #10 #if size_min is larger than size, the size is deterministic
config.dataset.ellipse_count    = 1 #10

In [13]:
print(config.dataset.blurred)

False


In [90]:
#test_transform = None
test_transform = T.Compose([
    T.Lambda(lambda x: T.Resize(size=64)(x))
])

In [ ]:
resized_imgs = [T.Resize(size=size)(X[0][0]) for size in (10, 20, 30, 64)]

resized_imgs_64x64 = []
for img in resized_imgs:
    resized_imgs_64x64.append(T.Resize(size=64)(img))

In [91]:
del training_dataset

In [92]:
training_dataset = _EllipsesDataset((640 if config.training_batch_count == -1 else config.training_batch_count)*config.training_batch_size, config.dataset.img_size, config.dataset.ellipse_count, config.dataset.ellipse_size, config.dataset.ellipse_size_min, transform=test_transform)

In [65]:
for newX in training_dataset:
    break

In [66]:
newX[0][0].size()

torch.Size([64, 64])

In [157]:
class EllipsesDataModule(pl.LightningDataModule):
    def __init__(self, config: omegaconf.DictConfig) -> None:
        super().__init__()
        self.config = config

    def train_dataloader(self) -> torch.utils.data.DataLoader:
        training_transform = torchvision.transforms.Compose([
            #torchvision.transforms.Lambda(lambda x: torchvision.transforms.functional.gaussian_blur(x, 5, 2.5))
            #torchvision.transforms.Lambda(lambda x: torchvision.transforms.Resize(size=64)(x))
            torchvision.transforms.Resize(size=[64,64])
        ])
        training_dataset = _EllipsesDataset((640 if self.config.training_batch_count == -1 else self.config.training_batch_count)*self.config.training_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, training_transform if self.config.dataset.blurred else None)
        return torch.utils.data.DataLoader(training_dataset, drop_last=self.config.drop_last_training_batch, batch_size=self.config.training_batch_size, shuffle=self.config.shuffle_training_data, num_workers=self.config.num_workers)
    
    def val_dataloader(self) -> torch.utils.data.DataLoader:
        validation_transform = torchvision.transforms.Compose([
            #torchvision.transforms.Lambda(lambda x: torchvision.transforms.functional.gaussian_blur(x, 5, 2.5))
            #torchvision.transforms.Lambda(lambda x: torchvision.transforms.Resize(size=64)(x))
            torchvision.transforms.Resize(size=64)
        ])
        validation_dataset = _EllipsesDataset((160 if self.config.validation_batch_count == -1 else self.config.validation_batch_count)*self.config.validation_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, validation_transform if self.config.dataset.blurred else None)
        return torch.utils.data.DataLoader(validation_dataset, drop_last=self.config.drop_last_validation_batch, batch_size=self.config.validation_batch_size, shuffle=self.config.shuffle_validation_data, num_workers=self.config.num_workers)

    def test_dataloader(self) -> torch.utils.data.DataLoader:
        test_transform = torchvision.transforms.Compose([
            #torchvision.transforms.Lambda(lambda x: torchvision.transforms.functional.gaussian_blur(x, 5, 2.5))
            #torchvision.transforms.Lambda(lambda x: torchvision.transforms.Resize(size=64)(x))
            torchvision.transforms.Resize(size=64)
        ])
        test_dataset = _EllipsesDataset((200 if self.config.test_batch_count == -1 else self.config.test_batch_count)*self.config.test_batch_size, self.config.dataset.img_size, self.config.dataset.ellipse_count, self.config.dataset.ellipse_size, self.config.dataset.ellipse_size_min, test_transform if self.config.dataset.blurred else None)
        return torch.utils.data.DataLoader(test_dataset, drop_last=self.config.drop_last_test_batch, batch_size=self.config.test_batch_size, shuffle=self.config.shuffle_test_data, num_workers=self.config.num_workers)

In [165]:
config.dataset.blurred = True

datamodule = EllipsesDataModule(config)

In [174]:
datamodule

In [183]:
train_loader = torch.utils.data.DataLoader(dataset=datamodule.train_dataloader(),
batch_size=1,
shuffle=True,
num_workers=0)

In [166]:
# trainloader = datamodule.train_dataloader()

In [184]:
len(datamodule.train_dataloader())

640

In [185]:
train_loader

In [197]:
#for batch_ndx, sample in enumerate(train_loader):
#    break

data_iter = iter(train_loader)
data_iter._next_data
#data = data_iter.next()

<bound method _SingleProcessDataLoaderIter._next_data of <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0x0000013C63CB85E0>>

In [186]:
x = iter(train_loader)

In [187]:
len(x)

640

In [188]:
for img in x:
    break

TypeError: 'DataLoader' object is not subscriptable

In [134]:
img[0].shape

torch.Size([32, 1, 64, 64])

In [126]:
for x in iter(trainloader):
    break

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\danil\Desktop\oct2022\py396_2022\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\danil\Desktop\oct2022\py396_2022\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\danil\Desktop\oct2022\py396_2022\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\danil\Desktop\oct2022\test\ellipses_datamodule.py", line 58, in __getitem__
    if self.ellipse_angle[idx] == 0.0:
IndexError: index 20451 is out of bounds for dimension 0 with size 20400


In [125]:
x.shape

AttributeError: 'list' object has no attribute 'shape'

In [10]:
import torchvision.transforms as T

In [12]:
training_dataset

In [ ]:
resized_imgs = [T.Resize(size=size)(X[0][0]) for size in (10, 20, 30, 64)]

In [34]:
orig_img = X[0][0]

In [35]:
def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()
    plt.savefig(f"my_resized.png")


In [41]:
resized_imgs = [T.Resize(size=size)(X[0][0]) for size in (10, 20, 30, 64)]

resized_imgs_64x64 = []
for img in resized_imgs:
    resized_imgs_64x64.append(T.Resize(size=64)(img))


NameError: name 'X' is not defined

In [54]:
fig, axs = plt.subplots(nrows=1, ncols=4, squeeze=False)

for cont, x in enumerate (resized_imgs):
    print(x[0].shape)
    ax = axs[0,cont]
    ax.imshow(x[0])
    
fig.savefig(f"my_resized.png")


torch.Size([10, 10])
torch.Size([20, 20])
torch.Size([30, 30])
torch.Size([64, 64])


In [56]:
fig, axs = plt.subplots(nrows=1, ncols=4, squeeze=False)

for cont, x in enumerate (resized_imgs_64x64):
    print(x[0].shape)
    ax = axs[0,cont]
    ax.imshow(x[0])
    
fig.savefig(f"my_resized_64x64.png")



torch.Size([64, 64])
torch.Size([64, 64])
torch.Size([64, 64])
torch.Size([64, 64])


<AxesSubplot:>

In [42]:

resized_imgs[0].shape

torch.Size([1, 10, 10])

In [32]:
(torch.full((my_img_count,), my_ellipses_count).to(torch.float32)).to(torch.int32)

tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.int32)

In [31]:
torch.poisson(torch.full((my_img_count,), my_ellipses_count).to(torch.float32)).to(torch.int32)

tensor([1, 1, 1,  ..., 0, 0, 2], dtype=torch.int32)

In [80]:
newX[0].shape

torch.Size([1, 64, 64])

In [73]:
test_transform = T.Compose([
    T.Lambda(lambda x: T.Resize(size=5)(x))
])

In [74]:
test_transform(newX[0]).shape

torch.Size([1, 5, 5])

In [86]:
test_transform(newX[0]).shape[-2:]

torch.Size([5, 5])

In [88]:
test_transform(newX[0]).view(test_transform(newX[0]).shape[-2:])

torch.Size([5, 5])

In [89]:
for cont, newX in enumerate(training_dataset):
    plt.imshow(test_transform(newX[0]).view(test_transform(newX[0]).shape[-2:]))
    plt.savefig(f"my_resized_one_ellipse_noPoisson__{cont:03}__10x10.png")
    if cont>10:
        break

In [21]:
# print(newX[0].shape)

cont = 1
for newX in training_dataset:
    plt.imshow((newX[0]).view(config.dataset.img_size,config.dataset.img_size))
    plt.savefig(f"my_ten_ellipse_noPoisson__{cont:03}.png")
    cont+=1
    if cont>10:
        break

In [67]:
torch.count_nonzero(X[0][0].view(256,256)[0:2,:])

tensor(0)

In [61]:
((X[0][31]).view(256,256))

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [10]:
config.training_batch_size

AttributeError: 'EllipsesDataModule' object has no attribute 'training_batch_size'

In [ ]:
# Execute training and testing
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", r"Checkpoint directory .+ exists and is not empty\.") #Needed thanks to the custom logger
    warnings.filterwarnings("ignore", r"The dataloader, ((train)|(val)|(test)) dataloader( \d+)?, does not have many workers which may be a bottleneck\. Consider increasing the value of the `num_workers` argument` \(try \d+ which is the number of cpus on this machine\) in the `DataLoader` init to improve performance\.") #BUG Contradictory warnings with num_workers on cluster and slow loading with LoDoPaB
    trainer = pytorch_lightning.Trainer(
        deterministic=config.deterministic, 
        callbacks=[pytorch_lightning.callbacks.ModelCheckpoint(dirpath=".")], 
        accelerator="gpu" if config.device == "cuda" else None, 
        max_epochs=config.epochs, 
        logger=CustomTensorBoardLogger(hydra.core.hydra_config.HydraConfig.get().runtime.output_dir, None, ""), 
        limit_train_batches=int(config.training_batch_count) if config.training_batch_count != -1 else len(datamodule.train_dataloader()), 
        limit_val_batches=int(config.validation_batch_count) if config.validation_batch_count != -1 else len(datamodule.val_dataloader()), 
        limit_test_batches=int(config.test_batch_count) if config.test_batch_count != -1 else len(datamodule.test_dataloader()))
    trainer.fit(model, datamodule)
    trainer.test(model, datamodule)

logging.getLogger("pytorch_lightning").handlers.remove(logger.root.handlers[1]) #Stops pytorch lightning from writing to previous runs during multiruns


this_dir = os.getcwd()
print(this_dir)

